## Hugging Face Tokenizer
### **1. 개요**

Hugging Face의 **Tokenizer**는 NLP 모델에서 텍스트를 처리할 때 **토큰화(Tokenization)** 작업을 수행하는 도구입니다.

텍스트를 **단어, 서브워드, 문장 등 다양한 단위로 변환**하며, 사전 학습된 토큰화 방식을 활용하여 **언어 모델의 입력 형식에 맞게 변환**합니다.

### **2. 주요 특징**

- **사전 학습된 모델 사용 가능** → BERT, GPT, T5 등 다양한 모델 지원
- **Fast Tokenizer 제공** → Rust 기반의 **빠른 토큰화(`FastTokenizer`)** 지원
- **다양한 토큰화 방식 지원** → WordPiece, Byte Pair Encoding(BPE), SentencePiece 등
- **특수 토큰 처리 가능** → `[CLS]`, `[SEP]`, `[MASK]`, `<pad>` 등의 토큰을 자동 추가
- **병렬 토큰화 지원** → 다중 문장 처리 속도 향상

### GPT2TokenizerFast

In [ ]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter

# GPT-2 모델의 토크나이저 로드
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# 데이터 파일 읽기
file_path = "../data/ai-terminology.txt"
with open(file_path, encoding="utf-8") as f:
    file_content = f.read()

print(" 원본 텍스트 미리보기:\n", file_content[:200])

# CharacterTextSplitter 설정 (Hugging Face 토크나이저 사용)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    hf_tokenizer,
    chunk_size=300,  # 각 청크 크기 (토큰 기준 아님)
    chunk_overlap=50,  # 청크 간 중복 부분
)

# 텍스트 분할 수행
split_texts = text_splitter.split_text(file_content)

# 분할된 텍스트 출력
print(f"\n 총 {len(split_texts)}개의 청크로 분할됨\n")
for i, chunk in enumerate(split_texts[:5]):  # 처음 5개만 출력
    print(f" Chunk {i+1} ({len(chunk)}자):\n{chunk}\n")

# 토크나이저로 텍스트를 토큰 단위로 변환하여 확인
tokenized_example = hf_tokenizer.tokenize(split_texts[0])
print(f"\n 첫 번째 청크의 토큰 개수: {len(tokenized_example)}")
print(" 첫 번째 청크의 토큰 리스트:", tokenized_example[:20])  # 앞 20개만 출력

In [6]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

# GPT-2 모델의 토크나이저 로드
print("GPT-2 토크나이저 로딩 중...")
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
print("토크나이저 로딩 완료")

# 데이터 파일 읽기
file_path = "../data/ai-terminology.txt"

# 파일 읽기
try:
    with open(file_path, encoding="utf-8") as f:
        file_content = f.read()
    print(f"파일 읽기 성공: {file_path}")
except Exception as e:
    print(f"파일 읽기 실패: {e}")
    exit()

# print("원본 텍스트 미리보기:")
# print("-" * 50)
# print(file_content[:200] + "...")
print(f"\n전체 텍스트 길이: {len(file_content)}자")

# ==========================================
# GPT-2 토크나이저 기본 정보
# ==========================================

print("\n" + "="*60)
print("GPT-2 토크나이저 기본 정보")
print("="*60)

print(f"모델명: {hf_tokenizer.name_or_path}")
print(f"어휘 크기: {hf_tokenizer.vocab_size:,}개")
print(f"최대 입력 길이: {hf_tokenizer.model_max_length:,} 토큰")

# 특수 토큰 확인
print(f"패딩 토큰: {hf_tokenizer.pad_token}")
print(f"시작 토큰: {hf_tokenizer.bos_token}")
print(f"끝 토큰: {hf_tokenizer.eos_token}")
print(f"언노운 토큰: {hf_tokenizer.unk_token}")

# 전체 텍스트의 토큰 개수 확인
total_tokens = len(hf_tokenizer.tokenize(file_content))
print(f"\n원본 텍스트 토큰 개수: {total_tokens:,}개")
print(f"문자 대 토큰 비율: {len(file_content)/total_tokens:.2f} (문자/토큰)")

# ==========================================
# 토크나이저 동작 확인
# ==========================================

print("\n" + "="*60)
print("토크나이저 동작 확인")
print("="*60)

# 예제 텍스트로 토크나이저 테스트
test_text = "RAG는 검색 기반의 텍스트 생성 모델입니다. Hello, AI!"
print(f"테스트 텍스트: {test_text}")

# 1. 토큰화 (tokenize)
tokens = hf_tokenizer.tokenize(test_text)
print(f"\n1. 토큰화 결과: {tokens}")
print(f"   토큰 개수: {len(tokens)}개")

# 2. 인코딩 (encode) - 토큰을 ID로 변환
token_ids = hf_tokenizer.encode(test_text)
print(f"\n2. 인코딩 결과: {token_ids}")
print(f"   토큰 ID 개수: {len(token_ids)}개")

# 3. 디코딩 (decode) - ID를 다시 텍스트로 변환
decoded_text = hf_tokenizer.decode(token_ids)
print(f"\n3. 디코딩 결과: {decoded_text}")

# 4. 개별 토큰 ID 확인
print(f"\n4. 토큰별 ID 매핑:")
for i, (token, token_id) in enumerate(zip(tokens, token_ids), 1):
    print(f"   {i:2d}. '{token}' -> {token_id}")

# ==========================================
# CharacterTextSplitter with HuggingFace 토크나이저
# ==========================================

print("\n" + "="*60)
print("CharacterTextSplitter with HuggingFace 토크나이저")
print("="*60)

# CharacterTextSplitter 설정 (Hugging Face 토크나이저 사용)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    hf_tokenizer,
    chunk_size=300,     # 토큰 개수 기준
    chunk_overlap=50,   # 토큰 단위 중복
    separator="\n\n"    # 구분자 설정
)

# 텍스트 분할 수행
split_texts = text_splitter.split_text(file_content)

print(f"총 {len(split_texts)}개의 청크로 분할됨")
print("\n처음 3개 청크 분석:")

for i, chunk in enumerate(split_texts[:3], 1):
    chunk_tokens = hf_tokenizer.tokenize(chunk)
    chunk_token_ids = hf_tokenizer.encode(chunk)
    
    print(f"\nChunk {i}:")
    print(f"  문자 수: {len(chunk):,}자")
    print(f"  토큰 수: {len(chunk_tokens):,}개")
    print(f"  토큰 ID 수: {len(chunk_token_ids):,}개")
    print(f"  내용 미리보기: {chunk[:100]}...")

# ==========================================
# 다양한 chunk_size 비교
# ==========================================

print("\n" + "="*60)
print("다양한 chunk_size 설정 비교")
print("="*60)

chunk_sizes = [100, 300, 500]

for chunk_size in chunk_sizes:
    print(f"\nchunk_size={chunk_size} 토큰:")
    
    splitter = CharacterTextSplitter.from_huggingface_tokenizer(
        hf_tokenizer,
        chunk_size=chunk_size,
        chunk_overlap=chunk_size//10,  # 10% 중복
        separator="\n\n"
    )
    
    chunks = splitter.split_text(file_content)
    
    # 통계 계산
    total_tokens = sum(len(hf_tokenizer.tokenize(chunk)) for chunk in chunks)
    avg_tokens = total_tokens / len(chunks) if chunks else 0
    
    print(f"  총 청크 수: {len(chunks)}개")
    print(f"  평균 토큰 수: {avg_tokens:.1f}개")
    print(f"  전체 토큰 수: {total_tokens:,}개")
    
    # 토큰 수 분포 확인
    token_counts = [len(hf_tokenizer.tokenize(chunk)) for chunk in chunks]
    min_tokens = min(token_counts) if token_counts else 0
    max_tokens = max(token_counts) if token_counts else 0
    
    print(f"  토큰 수 범위: {min_tokens}~{max_tokens}개")

# ==========================================
# 토크나이저별 비교 (GPT-2 vs tiktoken)
# ==========================================

print("\n" + "="*60)
print("토크나이저별 비교")
print("="*60)

# tiktoken과 비교 (OpenAI 토크나이저)
try:
    import tiktoken
    
    # OpenAI 토크나이저
    openai_encoding = tiktoken.get_encoding("cl100k_base")
    
    # 같은 텍스트로 비교
    comparison_text = "RAG는 검색 기반의 텍스트 생성 모델입니다. AI와 머신러닝을 활용합니다."
    
    # GPT-2 토큰화
    gpt2_tokens = hf_tokenizer.tokenize(comparison_text)
    gpt2_token_count = len(gpt2_tokens)
    
    # OpenAI 토큰화
    openai_tokens = openai_encoding.encode(comparison_text)
    openai_token_count = len(openai_tokens)
    
    print(f"비교 텍스트: {comparison_text}")
    print(f"\nGPT-2 토크나이저:")
    print(f"  토큰 수: {gpt2_token_count}개")
    print(f"  토큰: {gpt2_tokens}")
    
    print(f"\nOpenAI 토크나이저 (cl100k_base):")
    print(f"  토큰 수: {openai_token_count}개")
    print(f"  토큰 ID: {openai_tokens}")
    
    print(f"\n토큰 수 차이: {abs(gpt2_token_count - openai_token_count)}개")
    
except ImportError:
    print("tiktoken이 설치되지 않음. GPT-2 토크나이저만 사용합니다.")

# ==========================================
# 한국어 처리 성능 테스트
# ==========================================

print("\n" + "="*60)
print("한국어 처리 성능 테스트")
print("="*60)

korean_texts = [
    "안녕하세요",
    "인공지능과 머신러닝",
    "자연어 처리 기술",
    "딥러닝 모델 학습",
    "검색 기반 생성 모델"
]

print("한국어 텍스트별 토큰화 결과:")
for i, text in enumerate(korean_texts, 1):
    tokens = hf_tokenizer.tokenize(text)
    token_ids = hf_tokenizer.encode(text)
    
    print(f"\n{i}. '{text}'")
    print(f"   토큰: {tokens}")
    print(f"   토큰 수: {len(tokens)}개")
    print(f"   문자당 토큰: {len(tokens)/len(text):.2f}")

# ==========================================
# Document 객체와 함께 사용
# ==========================================

print("\n" + "="*60)
print("Document 객체와 함께 사용")
print("="*60)


# Document 객체 생성
document = Document(
    page_content=file_content,
    metadata={
        "source": "ai-terminology.txt",
        "type": "glossary",
        "tokenizer": "gpt2"
    }
)

# Document 분할
doc_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    hf_tokenizer,
    chunk_size=300,
    chunk_overlap=50
)

doc_chunks = doc_splitter.split_documents([document])

print(f"Document 분할 결과: {len(doc_chunks)}개 청크")

for i, doc_chunk in enumerate(doc_chunks[:2], 1):
    token_count = len(hf_tokenizer.tokenize(doc_chunk.page_content))
    print(f"\nDocument Chunk {i}:")
    print(f"  토큰 수: {token_count}개")
    print(f"  메타데이터: {doc_chunk.metadata}")
    print(f"  내용: {doc_chunk.page_content[:100]}...")

GPT-2 토크나이저 로딩 중...


Token indices sequence length is longer than the specified maximum sequence length for this model (5172 > 1024). Running this sequence through the model will result in indexing errors
Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300
Created a chunk of size 321, which is longer than the specified 100
Created a chunk of size 362, which is longer than the specified 100
Created a chunk of size 237, which is longer than the specified 100
Created a chunk of size 201, which is longer than the specified 100
Created a chunk of size 226, which is longer than the specified 100
Created a chunk of size 236, which is longer than the specified 100
Created a chunk of size 180, which is longer than the specified 100
Created a chunk of size 240, which is longer than the specified 100
Created a chunk of size 226, which is longer than the specified 100
Created a chunk of size 205, which is longer than the specified 100


토크나이저 로딩 완료
파일 읽기 성공: ../data/ai-terminology.txt

전체 텍스트 길이: 3036자

GPT-2 토크나이저 기본 정보
모델명: gpt2
어휘 크기: 50,257개
최대 입력 길이: 1,024 토큰
패딩 토큰: None
시작 토큰: <|endoftext|>
끝 토큰: <|endoftext|>
언노운 토큰: <|endoftext|>

원본 텍스트 토큰 개수: 5,172개
문자 대 토큰 비율: 0.59 (문자/토큰)

토크나이저 동작 확인
테스트 텍스트: RAG는 검색 기반의 텍스트 생성 모델입니다. Hello, AI!

1. 토큰화 결과: ['RAG', 'ë', 'Ĭ', 'Ķ', 'Ġ', 'ê', '²', 'Ģ', 'ì', 'ĥ', 'ī', 'Ġ', 'ê', '¸', '°', 'ë', '°', 'ĺ', 'ìĿ', 'ĺ', 'Ġ', 'í', 'ħ', 'į', 'ì', 'Ĭ', '¤', 'í', 'Ĭ', '¸', 'Ġì', 'ĥ', 'Ŀ', 'ì', 'Ħ', '±', 'Ġë', 'ª', '¨', 'ë', 'į', '¸', 'ì', 'ŀ', 'ħ', 'ëĭ', 'Ī', 'ëĭ', '¤', '.', 'ĠHello', ',', 'ĠAI', '!']
   토큰 개수: 54개

2. 인코딩 결과: [33202, 167, 232, 242, 220, 166, 110, 222, 168, 225, 231, 220, 166, 116, 108, 167, 108, 246, 35975, 246, 220, 169, 227, 235, 168, 232, 97, 169, 232, 116, 23821, 225, 251, 168, 226, 109, 31619, 103, 101, 167, 235, 116, 168, 252, 227, 46695, 230, 46695, 97, 13, 18435, 11, 9552, 0]
   토큰 ID 개수: 54개

3. 디코딩 결과: RAG는 검색 기반의 텍스트 생성 모델입니다. Hello, AI!

4. 토큰별 ID 매핑:
    1

Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300


비교 텍스트: RAG는 검색 기반의 텍스트 생성 모델입니다. AI와 머신러닝을 활용합니다.

GPT-2 토크나이저:
  토큰 수: 81개
  토큰: ['RAG', 'ë', 'Ĭ', 'Ķ', 'Ġ', 'ê', '²', 'Ģ', 'ì', 'ĥ', 'ī', 'Ġ', 'ê', '¸', '°', 'ë', '°', 'ĺ', 'ìĿ', 'ĺ', 'Ġ', 'í', 'ħ', 'į', 'ì', 'Ĭ', '¤', 'í', 'Ĭ', '¸', 'Ġì', 'ĥ', 'Ŀ', 'ì', 'Ħ', '±', 'Ġë', 'ª', '¨', 'ë', 'į', '¸', 'ì', 'ŀ', 'ħ', 'ëĭ', 'Ī', 'ëĭ', '¤', '.', 'ĠAI', 'ì', 'Ļ', 'Ģ', 'Ġë', '¨', '¸', 'ì', 'ĭ', 'ł', 'ë', 'Ł', '¬', 'ëĭ', 'Ŀ', 'ìĿ', 'Ħ', 'Ġ', 'í', 'Ļ', 'ľ', 'ì', 'ļ', '©', 'íķ', '©', 'ëĭ', 'Ī', 'ëĭ', '¤', '.']

OpenAI 토크나이저 (cl100k_base):
  토큰 수: 33개
  토큰 ID: [49, 1929, 16969, 86422, 78326, 55216, 39277, 246, 21028, 10997, 45204, 54289, 53017, 55170, 69697, 116, 80052, 13, 15592, 81673, 5251, 101, 116, 83628, 61394, 9019, 251, 18359, 47932, 250, 27797, 61938, 13]

토큰 수 차이: 48개

한국어 처리 성능 테스트
한국어 텍스트별 토큰화 결과:

1. '안녕하세요'
   토큰: ['ì', 'ķ', 'Ī', 'ë', 'ħ', 'ķ', 'íķ', 'ĺ', 'ì', 'Ħ', '¸', 'ì', 'ļ', 'Ķ']
   토큰 수: 14개
   문자당 토큰: 2.80

2. '인공지능과 머신러닝'
   토큰: ['ìĿ', '¸', 'ê', '³', 'µ', 'ì', '§', 'Ģ', 'ë', '

### AutoTokenizer (한국어 특화된 모델 - KoBERT)

In [5]:
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 한국어 지원 토크나이저 사용 (kcbert-base)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")  

# 예제 문장 (한글 포함)
text = "자연어 처리는 인공지능의 핵심 기술 중 하나입니다. NLP 모델은 의미를 이해하고 텍스트를 생성할 수 있습니다."

# 적절한 청크 크기 조정 (너무 작지 않게 설정)
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer, 
    chunk_size=20,  # 더 큰 청크 크기
    chunk_overlap=5  # 문맥 유지를 위한 오버랩
)

# 텍스트 분할
split_texts = splitter.split_text(text)

# 실행 결과 출력
print(f" 총 {len(split_texts)}개 청크로 분할됨\n")
for i, chunk in enumerate(split_texts):
    print(f" Chunk {i+1}: {chunk}\n")

# 첫 번째 청크의 토큰화 결과
tokens = tokenizer.tokenize(split_texts[0])
print(f" 첫 번째 청크의 토큰 개수: {len(tokens)}")
print(" 첫 번째 청크의 토큰 리스트:", tokens)

c:\Users\sogno\AppData\Local\pypoetry\Cache\virtualenvs\langchain-basic-farQSE-J-py3.12\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sogno\.cache\huggingface\hub\models--beomi--kcbert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


 총 2개 청크로 분할됨

 Chunk 1: 자연어 처리는 인공지능의 핵심 기술 중 하나입니다. NLP 모델은 의미를 이해하고

 Chunk 2: 모델은 의미를 이해하고 텍스트를 생성할 수 있습니다.

 첫 번째 청크의 토큰 개수: 20
 첫 번째 청크의 토큰 리스트: ['자연', '##어', '처리', '##는', '인공', '##지능', '##의', '핵심', '기술', '중', '하나', '##입니다', '.', 'N', '##L', '##P', '모델', '##은', '의미를', '이해하고']
